In [29]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from IPython.display import display, HTML
import re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time
from datetime import datetime # attach current date to export filename
import sys
import ast # convert str back to tuple/list/int, etc
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import string

# consider bbp all present first
# All/Pair/UniquePep/N-glycoPep all add sheet2 for color summary table
df = pd.read_excel('20210622_byonic_PTPRA_NoOG_T_pglyco_PTPRA_NoOG_T_All.xlsx', header = 0)
fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in df.columns]
df.columns = fixed_colname
# generate new glycan source/new fragment type for color summary table
df.loc[(df['ColorCode']=='lightgreen(#ccffcc)')|(df['ColorCode']=='normalgreen(#99ff99)')|(df['ColorCode']=='lightblue(#cce6ff)')|(df['ColorCode']=='normalblue(#99ccff)')|(df['ColorCode']=='lightorange(#ffebcc)')|(df['ColorCode']=='normalorange(#ffd699)'), 'new_GlycanSource'] = df['GlycanSource']
df.loc[(df['ColorCode']=='lightgreen(#ccffcc)')|(df['ColorCode']=='normalgreen(#99ff99)')|(df['ColorCode']=='lightorange(#ffebcc)')|(df['ColorCode']=='normalorange(#ffd699)'), 'new_FragmentType'] = df['Fragment\r\nType[Byonic]']
df.loc[(df['ColorCode']=='lightblue(#cce6ff)')|(df['ColorCode']=='normalblue(#99ccff)'), 'new_FragmentType'] = df['FragmentType[pGlyco]']
df.loc[(df['ColorCode']=='deepgreen(#66ff66)'), ['new_GlycanSource', 'new_FragmentType']] = 'N/A'
df.loc[(df['ColorCode']=='deepblue(#66b3ff)'), ['new_GlycanSource', 'new_FragmentType']] = 'N/A'
# add categories (Byonic/pGlyco/Overlap)
df.loc[df['ColorCode'].str.contains('green')== True, 'Category'] = 'Byonic'
df.loc[df['ColorCode'].str.contains('blue')== True, 'Category'] = 'pGlyco'
df.loc[df['ColorCode'].str.contains('orange')== True, 'Category'] = 'Overlap'
# splitting df
b_df = df.loc[(df['N-site(SequonBased)[Byonic]'] != '-1')&(df['N-site(SequonBased)[Byonic]'] != -1), ['N-site(SequonBased)[Byonic]', 'ColorCode', 'new_GlycanSource', 'new_FragmentType', 'Category']].drop_duplicates().reset_index(drop=True).rename(columns={'N-site(SequonBased)[Byonic]':'N-site(Byonic ∪ pGlyco) →'}) 
p_df = df.loc[(df['ProSites[pGlyco]'] != '-1')&(df['ProSites[pGlyco]'] != -1), ['ProSites[pGlyco]', 'ColorCode', 'new_GlycanSource', 'new_FragmentType', 'Category']].drop_duplicates().reset_index(drop=True).rename(columns={'ProSites[pGlyco]':'N-site(Byonic ∪ pGlyco) →'}) 
a1, a2 = b_df.align(p_df, join='outer', axis = 1) # align cols only
new_df = pd.concat([a1,a2]) # concat vertically 
new_df = new_df.drop_duplicates().reset_index(drop=True)
new_df.to_excel('new_df.xlsx', index = False)
new_df['cnt'] = new_df.groupby('ColorCode')['ColorCode'].transform('size')
new_df.to_excel('new_df_cnt.xlsx', index = False)
color_summary = pd.pivot_table(new_df, values='cnt', index=['Category', 'new_GlycanSource', 'new_FragmentType', 'ColorCode'], columns=['N-site(Byonic ∪ pGlyco) →'], aggfunc=np.sum)
# col sort by site
site_lst = color_summary.columns.tolist()
site_lst = [ast.literal_eval(i) if type(i) == str else i for i in site_lst]
site_lst.sort(key=lambda v: (isinstance(v, tuple), v))
color_summary = color_summary.reindex(columns=site_lst)
# rename index 
color_summary.index = color_summary.index.set_names(['Category', 'GlycanSource', 'FragmentType', 'ColorCode'])
color_summary.to_excel('color_summary_test.xlsx', index_label=[color_summary.index.name, color_summary.columns.name])